# Numerical Methods - Week 3
## Kiran Shila - U54532811

In this notebook, I will demonstrate the deliverables for the week 3 assignment.

## Gaussian Elimination
The first part of this assignment was to write gaussian elimination code that utilizes partial pivoting

In [1]:
using BenchmarkTools
using LinearAlgebra

In [2]:
function gauss_elim(A::Matrix,b::Vector)
    # Create augmented matrix
    # make sure its a float otherwise we get float errors
    n = size(b)[1]
    A_Aug = float([A b])
    # For every column except the last one because pivots
    for k = 1:n-1
        # Find the largest pivot in this column, referenced to the pivot
        location = findmax(broadcast(abs,A_Aug[k:end,k]))[2] + k - 1
        maxVal = A_Aug[location,k]
        # Check to see if current pivot is the max,
        # if it isn't - swap
        if A_Aug[k,k] != maxVal
            for j in k:n+1
                A_Aug[location,j], A_Aug[k,j] = A_Aug[k,j], A_Aug[location,j]
            end
        end
        # Now perform gaussian elimination
        # For every row under the pivot
        for i = k+1:n
            # Normalize to the pivot and subtract from pivot row
            if A_Aug[i,k] != 0 # Ensures that we need to perform elimination
                scale =  A_Aug[i,k] / A_Aug[k,k]
                for j in k:size(A_Aug)[2]
                    A_Aug[i,j] = A_Aug[i,j] - scale * A_Aug[k,j]
                end
            end
        end
    end
    # Now back substitute to get to x
    x = zeros(Float64,n) # Create zeros for solution vector
    x[n] = A_Aug[end,end] / A_Aug[n,n] # Set the starting x
    # For every row, working backwards starting with the second from the bottom
    for i in n-1:-1:1
        x[i] = (A_Aug[i,end] - sum( [x[j] * A_Aug[i,j] for j in i+1:n] )) / A_Aug[i,i]
    end
    # Return solution vector x and U
    return x, A_Aug[:,1:end-1]
end

gauss_elim (generic function with 1 method)

Now to test against the examples from the assignment:
\begin{equation*}
\mathbf{A} =  \begin{vmatrix}
1 & 2 & 1\\
3 & 8 & 1\\
0 & 4 & 1
\end{vmatrix}
\end{equation*}

\begin{equation*}
\mathbf{b} =  \begin{vmatrix}
2 \\ 12 \\ 2
\end{vmatrix}
\end{equation*}

In [3]:
A = [1 2 1; 3 8 1; 0 4 1]
b = [2;12;2]
x,U = gauss_elim(A,b)
x

3-element Array{Float64,1}:
  2.0
  1.0
 -2.0

This matches what was expected from the assignment. Just to compare to the built-in solver:

In [4]:
A\b

3-element Array{Float64,1}:
  2.0
  1.0
 -2.0

And checking the upper triangular that we used to perform back-substitution

In [5]:
U

3×3 Array{Float64,2}:
 3.0  8.0  1.0     
 0.0  4.0  1.0     
 0.0  0.0  0.833333

Trying out the other example:
\begin{equation*}
\mathbf{A} =  \begin{vmatrix}
2 & 6 & 10\\
1 & 3 & 3\\
3 & 14 & 28
\end{vmatrix}
\end{equation*}

\begin{equation*}
\mathbf{b} =  \begin{vmatrix}
0 \\ 2 \\ -8
\end{vmatrix}
\end{equation*}

In [6]:
A = [2 6 10;1 3 3;3 14 28]
b = [0;2;-8]
x,U = gauss_elim(A,b)
x

3-element Array{Float64,1}:
  2.0000000000000013
  0.9999999999999992
 -0.9999999999999998

In [7]:
U

3×3 Array{Float64,2}:
 3.0  14.0      28.0    
 0.0  -3.33333  -8.66667
 0.0   0.0      -2.0    

In [8]:
A\b

3-element Array{Float64,1}:
  2.0000000000000013
  0.9999999999999992
 -0.9999999999999998

Interestingly enough, our solver was just as unstable as the built-in. Lets compare time, though

In [9]:
@benchmark gauss_elim(A,b)

BenchmarkTools.Trial: 
  memory estimate:  1.78 KiB
  allocs estimate:  39
  --------------
  minimum time:     1.959 μs (0.00% GC)
  median time:      2.038 μs (0.00% GC)
  mean time:        2.822 μs (23.85% GC)
  maximum time:     5.456 ms (99.92% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [10]:
@benchmark A\b

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  7
  --------------
  minimum time:     653.646 ns (0.00% GC)
  median time:      714.885 ns (0.00% GC)
  mean time:        909.416 ns (13.00% GC)
  maximum time:     587.648 μs (99.78% GC)
  --------------
  samples:          10000
  evals/sample:     161

Our code sucks. It is about 4 time slower than the built-in solver and uses 3 times more RAM.

The last part was to compare with computing the inverse. Given

Trying out the other example:
\begin{equation*}
\mathbf{A} =  \begin{vmatrix}
1 & 2 & 1\\
3 & 8 & 1\\
0 & 4 & 1
\end{vmatrix}
\end{equation*}

\begin{equation*}
\mathbf{b} =  \begin{vmatrix}
2 \\ 12 \\ 2
\end{vmatrix}
\end{equation*}

First lets make sure the sovler works.

In [11]:
A = [1 2 1;3 8 1;0 4 1]
b = [2;12;2]
x, U = gauss_elim(A,b)
x - (A\b) == zeros(length(b)) # Is our result correct?

true

Now compare performace with built-ins and inverse

In [12]:
@benchmark gauss_elim(A,b)

BenchmarkTools.Trial: 
  memory estimate:  1.78 KiB
  allocs estimate:  39
  --------------
  minimum time:     1.979 μs (0.00% GC)
  median time:      2.190 μs (0.00% GC)
  mean time:        3.754 μs (35.68% GC)
  maximum time:     9.226 ms (99.94% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [13]:
@benchmark A\b

BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  4
  --------------
  minimum time:     338.491 ns (0.00% GC)
  median time:      355.933 ns (0.00% GC)
  mean time:        434.930 ns (13.15% GC)
  maximum time:     249.649 μs (99.80% GC)
  --------------
  samples:          10000
  evals/sample:     218

In [14]:
@benchmark A^-1*b

BenchmarkTools.Trial: 
  memory estimate:  2.36 KiB
  allocs estimate:  8
  --------------
  minimum time:     913.250 ns (0.00% GC)
  median time:      986.222 ns (0.00% GC)
  mean time:        1.408 μs (22.05% GC)
  maximum time:     1.405 ms (99.89% GC)
  --------------
  samples:          10000
  evals/sample:     36

So the `backslash` operator was still the best, but our code had better memory efficiency than the inverse. The inverse was still faster.

## LU Decomposition
Modified the gaussian elimination code to support LU decomposition

In [15]:
function my_lu(A::Matrix)
    # Create copy
    # make sure its a float otherwise we get float errors
    n = size(A)[1]
    A_Copy = float(A)
    L = float(one(A))
    P = one(A)
    # For every column except the last one because pivots
    for k = 1:n-1
        # Find the largest pivot in this column
        location = findmax(broadcast(abs,A_Copy[k:end,k]))[2] + k - 1
        maxVal = A_Copy[location,k]
        # Check to see if current pivot is the max,
        # if it isn't - swap
        if A_Copy[k,k] != maxVal
            for j in k:size(A_Copy)[2]
                temp = A_Copy[location,j]
                A_Copy[location,j] = A_Copy[k,j]
                A_Copy[k,j] = temp
            end
            # And swap in P
            P[location,:] , P[k,:] = P[k,:] , P[location,:]
            # And swap columns in L
            L[:,location] , L[:,k] = L[:,k] , L[:,location]
        end

        # Now perform gaussian elimination
        # For every row under the pivot
        for i = k+1:n
            # Normalize to the pivot and subtract from pivot row
            if A_Copy[i,k] != 0 # Ensures that we need to perform elimination
                scale =  A_Copy[i,k] / A_Copy[k,k]
                thisL = float(one(A))
                thisL[i,k] = scale
                L = L * thisL
                for j in k:size(A_Copy)[2]
                    A_Copy[i,j] = A_Copy[i,j] - scale * A_Copy[k,j]
                end
            end
        end
    end
    # Return solution vector L and U
    return P*L,A_Copy,P
end

my_lu (generic function with 1 method)

And now test

In [16]:
A = [2 6 10;1 3 3;3 14 28]
L,U,P = my_lu(A)
display(L)
display(U)
display(P)

3×3 Array{Float64,2}:
 1.0       0.0  0.0
 0.666667  1.0  0.0
 0.333333  0.5  1.0

3×3 Array{Float64,2}:
 3.0  14.0      28.0    
 0.0  -3.33333  -8.66667
 0.0   0.0      -2.0    

3×3 Array{Int64,2}:
 0  0  1
 1  0  0
 0  1  0

In [17]:
isapprox(L*U,P*A)

true

In [18]:
_L,_U = lu(A)
isapprox(_L,L) & isapprox(_U,U) # Both need to be true

true

## QR Decomposition
Just testing built-ins. Julia doesn't natively have magic() so I am just going to use randoms.

In [19]:
N = 3
A = rand(N,N)
Q,R = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.341519   0.617561   0.708508
 -0.709807   0.32465   -0.625121
 -0.616067  -0.716395   0.327475
R factor:
3×3 Array{Float64,2}:
 -1.13797  -0.980976  -1.14548 
  0.0       0.834652  -0.125268
  0.0       0.0        0.604019

In [20]:
isapprox(Q*R,A) # Using approx for stability reasons

true

Hey look its a normal decomposition

## Eigenvalues
Just testing built-ins. Julia doesn't natively have magic() so I am just going to use randoms.

In [21]:
N = 3
A = rand(N,N)
eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 1.608184016825046  
 0.18042200078145032
 0.4184987119764079 
eigenvectors:
3×3 Array{Float64,2}:
 0.478262   0.379229  -0.154369
 0.312082  -0.717756  -0.168049
 0.820896   0.583962   0.973617

Neat.